In [1]:
import cobra
import libsbml
from cobra.core import Metabolite, Reaction
import pandas as pd
from cobra import flux_analysis
ModelF = cobra.io.read_sbml_model("CAM_12PModel.xml")
ModelF.solver="glpk"
cobra.flux_analysis.pfba(ModelF)

#adding maintenance cost
PPFD = 100                            #light intensity of the model
ATPase = (0.0049*PPFD) + 2.7851       #non-growth assocaited maintenance (NGAM) cost based on light - see Topfer et al 2020 Supplemental information section 1.2.3

for i in range(1,13):
    ModelF.reactions.get_by_id("ATPase_tx"+str(i)).lower_bound = ATPase
    ModelF.reactions.get_by_id("ATPase_tx"+str(i)).upper_bound = ATPase
    
    #Setting NADPH demand to 1/3 of ATP demand and distributing this demand to cytosol, plastid and mitochondria based on Cheung et al 2013 (doi: 10.1111/tpj.12252)
    ModelF.reactions.get_by_id("NADPHoxc_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxc_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF.reactions.get_by_id("NADPHoxp_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxp_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF.reactions.get_by_id("NADPHoxm_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxm_tx"+str(i)).upper_bound = ATPase/9
        
cobra.flux_analysis.pfba(ModelF)

,fluxes,reduced_costs
PRO_PROTON_vc1,0.000000,2.0
Ca_tx1,0.000000,2.0
H2O_xc1,0.000000,2.0
sCIT_biomass1,0.000000,2.0
ACETYLGLUTKIN_RXN_p1,0.000108,-2.0
...,...,...
THREO_DS_ISO_CITRATE_v9_accumulation,0.299252,-2.0
THREO_DS_ISO_CITRATE_v10_accumulation,0.299252,-2.0
THREO_DS_ISO_CITRATE_v11_accumulation,0.299252,-2.0
THREO_DS_ISO_CITRATE_v12_accumulation,0.299252,-2.0


In [2]:
ModelF3 = ModelF.copy()

In [3]:
#Apply constraints
#Constrain ATP_ADP_Pi_pc to 0 (we know NTT, plastidic nucleotide transporter is only active in importing ATP to chloroplast at night or in non-photosynthetic tissues (Reinhold et al., 2007; Flugge et al., 2011; Voon and Lim, 2019))
for i in range(1,7):    
    ModelF3.reactions.get_by_id("ATP_ADP_Pi_pc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("ATP_ADP_Pi_pc"+str(i)).upper_bound = 0
    
#Constrain PEPCK to 0 (because we are modelling Phalaenopsis) (ME takes over as decarboxylating enzyme)    
for i in range(1,13):    
    ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).upper_bound = 0

#Remove reaction H_pc (because this was included as a hypothetical reaction)
for i in range(1,13):    
    ModelF3.reactions.get_by_id("H_pc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("H_pc"+str(i)).upper_bound = 0

#Constrain a flux ratio between NAD-ME and NADP-ME according to temporal enzyme activity measurements in Phalaenopsis, Kalanchoë and Dever et al. 2015
for i in range(1,13):
    new_constraint1 = ModelF3.problem.Constraint(
        8*(ModelF3.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i)).flux_expression + ModelF3.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i)).flux_expression) - ModelF3.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint1)
    
#Constrain a flux ratio between PPDK_c and PPDK_p according to Kondo et al. 2000 and Dever et al. 2015
for i in range(1,13):
    new_constraint2 = ModelF3.problem.Constraint(
        2*ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i)).flux_expression - ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint2)
    
#Change direction of mitochondrial PYR-H+ symporter (PiC) allowing only PYR import into mitonchondria (Le et al., 2021)
for i in range(1,13):    
    ModelF3.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i)).lower_bound = -1000
    ModelF3.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i)).upper_bound = 0
    
#Add a PYR channel reaction to re-allow PYR export from mitochondria
for i in range(1,13):
    rxn = Reaction("PYR_mc"+str(i)+"_channel",name = "PYR_mc"+str(i)+"_channel")
   
    rxn.add_metabolites({ModelF3.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
                         ModelF3.metabolites.get_by_id("PYRUVATE_c"+str(i)):1})
   
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    ModelF3.add_reactions([rxn,])
    
sol3 = flux_analysis.parsimonious.pfba(ModelF3)

In [4]:
sol3 = flux_analysis.parsimonious.pfba(ModelF3)
cobra.io.write_sbml_model(ModelF3, "CAM_12P_ME_Model.xml")

In [5]:
from Functions import checkProtonFluxes
checkProtonFluxes(ModelF3, tag="ME")

In [6]:
#write csv file with all reactions of all model phases included
fout = open("pFBA_output_PiCactive.csv","w")

for rxn in ModelF3.reactions:
    fout.write(rxn.id+","+rxn.name+","+rxn.reaction+","+str(rxn.flux)+"\n")
    
fout.close()

In [7]:
#Additional checks that are reported in the manuscript

ModelF4 = ModelF3.copy()

# Check importance of the PiC transporter 
for i in range(1,13):
    ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).upper_bound = 0
    
#Also constrain reaction H_mc to carry zero flux in this case (because this was included as a hypothetical reaction)
for i in range(1,13):    
    ModelF4.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0

sol4 = flux_analysis.parsimonious.pfba(ModelF4)

#write csv file with all reactions of all model phases included
fout = open("pFBA_output_PiCoff.csv","w")

for rxn in ModelF4.reactions:
    fout.write(rxn.id+","+rxn.name+","+rxn.reaction+","+str(rxn.flux)+"\n")
    
fout.close()

In [8]:
#Simulate different mitochondrial pyruvate export mechanisms (symport and antiport)
#Uncomment one to simulate, leave the other commented
ModelF4 = ModelF3.copy()

#Reactivate PiC if set to 0 in previous code block
# for i in range(1,13):
#     ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).lower_bound = -1000
#     ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).upper_bound = 1000
    
#PYR-H symport reaction
# for i in range(1,13):
#     rxn = Reaction("PYR_H_mc"+str(i)+"_symport",name = "PYR_H_mc"+str(i)+"_symport")
   
#     rxn.add_metabolites({ModelF4.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PROTON_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PYRUVATE_c"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PROTON_c"+str(i)):1})
   
#     rxn.lower_bound = 0
#     rxn.upper_bound = 1000
#     ModelF4.add_reaction(rxn)
#     #turn off PYR channel
#     ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel").upper_bound = 0
#     ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel").lower_bound = 0

#PYR-H antiport reaction
# for i in range(1,13):
#     rxn = Reaction("PYR_H_mc"+str(i)+"_antiport",name = "PYR_H_mc"+str(i)+"_antiport")
   
#     rxn.add_metabolites({ModelF4.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PROTON_m"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PYRUVATE_c"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PROTON_c"+str(i)):-1})
   
#     rxn.lower_bound = 0
#     rxn.upper_bound = 1000
#     ModelF4.add_reaction(rxn)
#     #turn off PYR channel
#     ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel").upper_bound = 0
#     ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel").lower_bound = 0

#Also constrain reaction H_mc to carry zero flux in this case (because this was included as a hypothetical reaction)
for i in range(1,13):    
    ModelF4.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0
    
sol4 = flux_analysis.parsimonious.pfba(ModelF4)

In [9]:
#Summary of fluxes through particular CAM reactions when checking different mitochondrial pyruvate export mechanisms
#Night-time fluxes 
print("Night-time fluxes:")

PEPC=0
for i in range(7,13):
    rxn = ModelF4.reactions.get_by_id("PEPCARBOX_RXN_c"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    PEPC=PEPC+totalFlux
print("---------------")
print("Total flux through PEPC: "+str(PEPC))

MalImp=0
for i in range (7,13):  
    rxn = ModelF4.reactions.get_by_id("MAL_PROTON_vc"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    MalImp=MalImp+totalFlux
print("---------------")
print("Total vacuolar malate influx: "+str(MalImp))

MPCnight=0
for i in range(7,13):
    rxn = ModelF4.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
    #print(rxn.reaction)
    #print(rxn.flux)
    MPCnight=MPCnight+totalFlux
print("---------------")
print("Total flux through MPC during the night: "+str(MPCnight))

PiCnight=0
for i in range(7,13):
    rxn = ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    PiCnight=PiCnight+totalFlux
print("---------------")
print("Total flux through PiC during the night: "+str(PiCnight))

ATPnightMito = 0
for i in range(7,13):
    rxn = ModelF4.reactions.get_by_id("Mitochondrial_ATP_Synthase_m"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
    ATPnightMito += totalFlux
print("---------------")
print("Total flux through mitochondrial ATP synthase during the night: " +str(ATPnightMito))

GAPDHnight=0
for i in range(7,13):
    rxn = ModelF4.reactions.get_by_id("GAPOXNPHOSPHN_RXN_c"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    GAPDHnight=GAPDHnight+totalFlux
print("---------------")
print("Total flux through cytosolic GAPDH during the night: "+str(GAPDHnight))

#uncomment one of the mechanisms to check flux
# PYRsymportNight=0
# for i in range(7,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_H_mc"+str(i)+"_symport")
#     totalFlux=(rxn.flux*60*60*2)/1000
# # #     print(rxn.reaction)
# # #     print(rxn.flux)
#     PYRsymportNight=PYRsymportNight+totalFlux
# print("---------------")
# print("Total flux through PYR/H+ symporter during the night: "+str(PYRsymportNight))

# PYRantiportNight=0
# for i in range(7,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_H_mc"+str(i)+"_antiport")
#     totalFlux=(rxn.flux*60*60*2)/1000
#     #print(rxn.reaction)
#     #print(rxn.flux)
#     PYRantiportNight=PYRantiportNight+totalFlux
# print("---------------")
# print("Total flux through PYR/H+ antiporter during the night: "+str(PYRantiportNight))

# PYRchannelNight=0
# for i in range(7,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel")
#     totalFlux=(rxn.flux*60*60*2)/1000
# #     print(rxn.reaction)
# #     print(rxn.flux)
#     PYRchannelNight=PYRchannelNight+totalFlux
# print("---------------")
# print("Total flux through PYR channel during the night: "+str(PYRchannelNight))

print("---------------")

#Day-time fluxes
print("Day-time fluxes:")

MalateEfflux=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("MAL_PROTON_rev_vc"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    MalateEfflux=MalateEfflux+totalFlux
print("---------------")
print("Total vacuolar malate efflux: "+str(MalateEfflux))

NADME=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    NADME=NADME+totalFlux
print("---------------")
print("Total flux through mitochondrial NAD-ME: "+str(NADME))

NADPMEcyt=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    NADPMEcyt=NADPMEcyt+totalFlux
print("---------------")
print("Total flux through cytosolic NADP-ME: "+str(NADPMEcyt))

NADPMEplast=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    NADPMEplast=NADPMEplast+totalFlux
print("---------------")
print("Total flux through plastidial NADP-ME: "+str(NADPMEplast))

PPDKcyt=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    PPDKcyt=PPDKcyt+totalFlux
print("---------------")
print("Total flux through cytosolic PPDK: "+str(PPDKcyt))

PPDKplast=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    PPDKplast=PPDKplast+totalFlux
print("---------------")
print("Total flux through plastidial PPDK: "+str(PPDKplast))

RubOxy=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("RXN_961_p"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    RubOxy=RubOxy+totalFlux
print("---------------")
print("Total flux through Rubisco oxygenase: "+str(RubOxy))

RubCarboxy=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    RubCarboxy=RubCarboxy+totalFlux
print("---------------")
print("Total flux through Rubisco carboxylase: "+str(RubCarboxy))

starch = 0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("GLYCOGENSYN_RXN_p"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
    #print(rxn.reaction)
    #print(rxn.flux)
    starch = starch+totalFlux
print("---------------")
print("Total flux through starch synthase: "+str(starch))

MPCday=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
    #print(rxn.reaction)
    #print(rxn.flux)
    MPCday=MPCday+totalFlux
print("---------------")
print("Total flux through MPC during the day: "+str(MPCday))

PiCday=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    PiCday=PiCday+totalFlux
print("---------------")
print("Total flux through PiC during the day: "+str(PiCday))

ATPdayMito = 0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("Mitochondrial_ATP_Synthase_m"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
    ATPdayMito += totalFlux
print("---------------")
print("Total flux through mitochondrial ATP synthase during the day: " +str(ATPdayMito))

GAPDHday=0
for i in range(1,7):
    rxn = ModelF4.reactions.get_by_id("GAPOXNPHOSPHN_RXN_c"+str(i))
    totalFlux=(rxn.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    GAPDHday=GAPDHday+totalFlux
print("---------------")
print("Total flux through cytosolic GAPDH during the day: "+str(GAPDHday))

#uncomment one of the mechanisms to check flux
# PYRsymportDay=0
# for i in range(1,7):
#     rxn = ModelF4.reactions.get_by_id("PYR_H_mc"+str(i)+"_symport")
#     totalFlux=(rxn.flux*60*60*2)/1000
# # #     print(rxn.reaction)
# # #     print(rxn.flux)
#     PYRsymportDay=PYRsymportDay+totalFlux
# print("---------------")
# print("Total flux through PYR/H+ symporter during the day: "+str(PYRsymportDay))

# PYRantiportDay=0
# for i in range(1,7):
#     rxn = ModelF4.reactions.get_by_id("PYR_H_mc"+str(i)+"_antiport")
#     totalFlux=(rxn.flux*60*60*2)/1000
#     #print(rxn.reaction)
#     #print(rxn.flux)
#     PYRantiportDay=PYRantiportDay+totalFlux
# print("---------------")
# print("Total flux through PYR/H+ antiporter during the day: "+str(PYRantiportDay))

# PYRchannelDay=0
# for i in range(1,7):
#     rxn = ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel")
#     totalFlux=(rxn.flux*60*60*2)/1000
# #     print(rxn.reaction)
# #     print(rxn.flux)
#     PYRchannelDay=PYRchannelDay+totalFlux
# print("---------------")
# print("Total flux through PYR channel during the day: "+str(PYRchannelDay))

phloem_rxn = ModelF4.reactions.get_by_id("Diel_phloem_export")
totalFlux=(phloem_rxn.flux*60*60*2)/1000
print("---------------")
print("Diel phloem export flux: " +str(totalFlux))

Night-time fluxes:
---------------
Total flux through PEPC: 181.6517257983019
---------------
Total vacuolar malate influx: 162.00405236122688
---------------
Total flux through MPC during the night: -22.152105211694536
---------------
Total flux through PiC during the night: -213.97838174247417
---------------
Total flux through mitochondrial ATP synthase during the night: 70.46985521941062
---------------
Total flux through cytosolic GAPDH during the night: 203.64000618931323
---------------
Day-time fluxes:
---------------
Total vacuolar malate efflux: 170.87925491647422
---------------
Total flux through mitochondrial NAD-ME: 160.71283516644024
---------------
Total flux through cytosolic NADP-ME: 20.089104395805027
---------------
Total flux through plastidial NADP-ME: 0.0
---------------
Total flux through cytosolic PPDK: 120.40111299841861
---------------
Total flux through plastidial PPDK: 60.200556499210805
---------------
Total flux through Rubisco oxygenase: 0.0
------------

In [10]:
#Check fluxes through reactions that re-export mitochondrial protons to the cytosol at the cost of reducing power when a PYR/H+ antiport mechanism is applied
Re_exp1=0
for i in range(1,7):
    rxn1= ModelF4.reactions.get_by_id("NADH_DEHYDROG_A_RXN_mc"+str(i))
    totalFlux1=(rxn1.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    Re_exp1=Re_exp1+totalFlux1
print("---------------")
print("Total flux through reaction1 that re-export mitochondrial protons to the cytosol at the cost of reducing power when a PYR/H+ antiport mechanism is applied: "+str(Re_exp1))

Re_exp2=0
for i in range(1,7):
    rxn2= ModelF4.reactions.get_by_id("1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mc"+str(i))
    totalFlux2=(rxn2.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    Re_exp2=Re_exp2+totalFlux2
print("---------------")
print("Total flux through reaction2 that re-export mitochondrial protons to the cytosol at the cost of reducing power when a PYR/H+ antiport mechanism is applied: "+str(Re_exp2))

Re_exp3=0
for i in range(1,7):
    rxn3= ModelF4.reactions.get_by_id("CYTOCHROME_C_OXIDASE_RXN_mc"+str(i))
    totalFlux3=(rxn3.flux*60*60*2)/1000
#     print(rxn.reaction)
#     print(rxn.flux)
    Re_exp3=Re_exp3+totalFlux3
print("---------------")
print("Total flux through reaction3 that re-export mitochondrial protons to the cytosol at the cost of reducing power when a PYR/H+ antiport mechanism is applied: "+str(Re_exp3))

---------------
Total flux through reaction1 that re-export mitochondrial protons to the cytosol at the cost of reducing power when a PYR/H+ antiport mechanism is applied: 0.0
---------------
Total flux through reaction2 that re-export mitochondrial protons to the cytosol at the cost of reducing power when a PYR/H+ antiport mechanism is applied: 0.0
---------------
Total flux through reaction3 that re-export mitochondrial protons to the cytosol at the cost of reducing power when a PYR/H+ antiport mechanism is applied: 0.0
